Collect 1 brand and see how the data inside that brand is

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("jgranizo/sentimental")
model = AutoModelForSequenceClassification.from_pretrained("jgranizo/sentimental")



c:\Users\grani\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Unrecognized model in jgranizo/sentimental. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, fsmt, funnel, fuyu, gem

In [43]:
import praw
from datetime import datetime, timedelta

# Initialize Reddit API client with your credentials
reddit = praw.Reddit(
    client_id='ZPr2EshdHCAQUHQ1B4LRdQ',
    client_secret='D_pSNQdQpLFu0xkme74Sket1zGUorg',
    user_agent='BrandDataCollector/1.0 by u/jgran'
)


# Define criteria
brand_name = "Apple"
min_score = 100  # Minimum score to filter high-upvote posts
min_comments = 50  # Minimum number of comments for high engagement
days_limit = 7  # Only get posts from the last 7 days

# Calculate the Unix timestamp for the date limit
date_limit = datetime.utcnow() - timedelta(days=days_limit)
timestamp_limit = date_limit.timestamp()

# Search for posts mentioning the brand in relevant subreddits
subreddits = ["technology", "gadgets", "stocks"]
posts_data = []
subreddit_dictionary = {}

for subreddit_name in subreddits:
    print(f"\nFetching posts about `{brand_name}` in r/{subreddit_name}")
    subreddit = reddit.subreddit(subreddit_name)
    
    for post in subreddit.search(brand_name, limit=10):  # Adjust limit as needed
        # Filter posts based on criteria
        if (post.score >= min_score and
            post.num_comments >= min_comments and
            post.created_utc >= timestamp_limit):

            # Store post attributes in a dictionary
            post_info = {
                "Title": post.title,
                "Score": post.score,
                "Comments": post.num_comments,
                "Created": datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                "Subreddit": post.subreddit.display_name,
                "Author": post.author,
                "URL": post.url,
                "Content": post.selftext  # Short preview of the post content
            }
             # Set comment sort order to 'top' and fetch top comments
            post.comment_sort = 'top'
            post.comments.replace_more(limit=0)  # Avoids collapsed comments
            top_comments = [comment.body for comment in post.comments[:5]]

            # Attach top comments to post_info
            post_info["Top_Comments"] = top_comments
         
            # Display each post as a "card"
            print("\n--- Post Card ---")
            print(f"Title: {post_info['Title']}")
            print(f"Score: {post_info['Score']} | Comments: {post_info['Comments']}")
            print(f"Created: {post_info['Created']} | Subreddit: {post_info['Subreddit']}")
            print(f"Author: {post_info['Author']}")
            print(f"URL: {post_info['URL']}")
            print(f"Content Preview: {post_info['Content']}")
            print("\nTop Comments:")
            for i, comment in enumerate(post_info["Top_Comments"], 1):
                print(f"Comment {i}: {comment}")  # Show first 100 characters for brevity
                if post_info['Title'] not in subreddit_dictionary:
                    subreddit_dictionary[post_info['Title']] = {}
                subreddit_dictionary[post_info['Title']][comment] = 0
            # Store post and comments for further analysis
            posts_data.append(post_info)
            




C:\Users\grani\AppData\Local\Temp\ipykernel_14824\3768352234.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_limit = datetime.utcnow() - timedelta(days=days_limit)



Fetching posts about `Apple` in r/technology

Fetching posts about `Apple` in r/gadgets

Fetching posts about `Apple` in r/stocks

--- Post Card ---
Title: Warren Buffett’s Berkshire Hathaway sold ~$14 billion worth of Apple this quarter. Cash pile hits record ~$325 billion.
Score: 1010 | Comments: 328
Created: 2024-11-02 08:19:31 | Subreddit: stocks
Author: themagicalpanda
URL: https://www.reddit.com/r/stocks/comments/1ghv5qo/warren_buffetts_berkshire_hathaway_sold_14/
Content Preview: Berkshires cash on hand hits a record $325 Billion

Currently owns ~$69.9 Billion worth of Apple $AAPL as of the end of Q3 down from $84.2B worth as of the end of Q2

No brk.a or brk.b shares repurchase in Q3

Posts Q3 revenue of $92.9B compared to $93.2B YoY

Operating earnings fell 6% from a year earlier to $10.09 billion as insurance underwriting earnings slumped

#

>**It is being widely reported that Berkshire's operating earnings declined 6.2% in the quarter. No. The reported figures include unre

In [55]:
print(subreddit_dictionary)

for title in subreddit_dictionary.keys():
    print("This is a subreddit title : ",title)
    for comment in subreddit_dictionary[title]:
       print("\nThis is a comment\n",comment,subreddit_dictionary[title][comment],"\n" )

{'Warren Buffett’s Berkshire Hathaway sold ~$14 billion worth of Apple this quarter. Cash pile hits record ~$325 billion.': {'His cash position is about 2x Blackrock market cap.': 0, 'Imagine having so much cash you could literally set a few million dollars on fire and no one would notice.': 0, 'What is Buffett cooking?': 0, 'That is a lot of cash.\n\nSo much speculation from succession, new administration, new tax fears to market being overvalued.\n\nBerkshire has been acquiring more insurance companies and buying stocks like CB which is also insurance.\n\nAlso Berkshire energy is growing as well.': 0, 'And AAPL bought back 95b worth of shares this year': 0}}
This is a subreddit title :  Warren Buffett’s Berkshire Hathaway sold ~$14 billion worth of Apple this quarter. Cash pile hits record ~$325 billion.

This is a comment
 His cash position is about 2x Blackrock market cap. 0 


This is a comment
 Imagine having so much cash you could literally set a few million dollars on fire and 

In [20]:
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import re

In [32]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
checkpoint = torch.load('checkpoint.pth', map_location='cpu')

model.load_state_dict(checkpoint['model_state_dict'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\grani\AppData\Local\Temp\ipykernel_14824\2737553353.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializ

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [60]:
def predict_sentence(sentence, model, tokenizer, label_mapping, max_len=128):
    # Tokenize and encode the sentence
    inputs = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    # Move tensors to the appropriate device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Set model to evaluation mode
    model.eval()

    # Run inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get the predicted class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map the predicted class to label
    predicted_label = label_mapping[predicted_class]

    return predicted_label

# Example usage
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}  # Adjust as needed
sentence = "Tech companies spent all this money on AI only to realize after they have no idea how to actually make money off it."


for title in subreddit_dictionary.keys():
    print("This is a subreddit title : ",title)
    print("This is the sentimental value of subreddit title: ",predict_sentence(title, model, tokenizer, label_mapping))
    for comment in subreddit_dictionary[title]:
       current_sentimental_value=predict_sentence(comment, model, tokenizer, label_mapping)
       subreddit_dictionary[title][comment] = current_sentimental_value
       print("\nThis is a comment\n",comment,"\n Its sentimental value is: ",predict_sentence(comment, model, tokenizer, label_mapping) ,"\n")






This is a subreddit title :  Warren Buffett’s Berkshire Hathaway sold ~$14 billion worth of Apple this quarter. Cash pile hits record ~$325 billion.
This is the sentimental value of subreddit title:  Negative

This is a comment
 His cash position is about 2x Blackrock market cap. 
 Its sentimental value is:  Neutral 


This is a comment
 Imagine having so much cash you could literally set a few million dollars on fire and no one would notice. 
 Its sentimental value is:  Neutral 


This is a comment
 What is Buffett cooking? 
 Its sentimental value is:  Negative 


This is a comment
 That is a lot of cash.

So much speculation from succession, new administration, new tax fears to market being overvalued.

Berkshire has been acquiring more insurance companies and buying stocks like CB which is also insurance.

Also Berkshire energy is growing as well. 
 Its sentimental value is:  Positive 


This is a comment
 And AAPL bought back 95b worth of shares this year 
 Its sentimental value is